In [1]:
!pip install imageio
!pip install keras

In [32]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import os, tarfile
import imageio
import tensorflow as tf
from scipy.io import loadmat
# from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

print(tf.test.gpu_device_name())


import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator

/device:GPU:0


In [0]:
fmnist_TRAIN_URL = 'https://www.dropbox.com/s/ynakalalovlqi5j/fashion-mnist_train.csv?dl=1'
fmnist_TEST_URL = 'https://www.dropbox.com/s/tado86497czrtlx/fashion-mnist_test.csv?dl=1'

# Utility functions

#### ToDos
- Create a function to fetch data from a url.
- Check if it is already downloaded.
- Check if the file is csv or tar gz etc.
- Add cross-validation code to be able to use sklearn cross_val_score function to quickly evaluate the performance.

In [0]:
def fetch_data(URL, DOWNLOAD_FOLDER, DOWNLOAD_FILE):
    
    if not os.path.isdir(DOWNLOAD_FOLDER):
        os.makedirs(DOWNLOAD_FOLDER)
        
    if not os.path.isfile(DOWNLOAD_FOLDER+DOWNLOAD_FILE):
        print('Beginning file download...')
        urllib.request.urlretrieve(URL, DOWNLOAD_FOLDER+DOWNLOAD_FILE)
        print('Done.')
  

In [0]:
def split_train_test(XY, n_splits=1, test_size=0.2, random_state=42):
    split = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    for train_index, test_index in split.split(XY[0], XY[1]):
        X_train, Y_train = XY[0][train_index,:], XY[1][train_index]
        X_test, Y_test = XY[0][test_index,:], XY[1][test_index]
        
    return X_train, Y_train, X_test, Y_test

In [0]:
def get_fmnist_data(url, download_folder, download_file, split_data=False):
    
    fetch_data(url, download_folder, download_file)
    XY = np.loadtxt(download_folder+download_file, dtype=np.float32, delimiter=',', skiprows=1)
    X = XY[:,1:]
    Y = XY[:,0]
    
    X = np.array(X) / 255.0
    
    #X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)

    if split_data:
        return split_train_test((X_new, Y), n_splits=1, test_size=0.2, random_state=42)
    
    n_samples = X.shape[0]
    X = np.reshape(X,(n_samples, 28, 28, 1))

    return X, np.expand_dims(Y,1)

In [7]:
labels = np.arange(0,10,1)
print(labels[1:])

[1 2 3 4 5 6 7 8 9]


In [0]:
def one_hot_encoder(label):
    
    encoder = OneHotEncoder(dtype=np.float32)
    label_1hot = encoder.fit_transform(label.reshape(-1,1))
    print('The labels are: {}'.format(np.unique(label)))
    return label_1hot

# Load data

In [0]:
root_folder = 'drive/app/fmnist/'
# root_folder = 'D:/dev/data/'

In [10]:
X_train, Y_train = get_fmnist_data(fmnist_TRAIN_URL, root_folder, 'fashion-mnist_train.csv',
                                split_data=False)

X_test, Y_test = get_fmnist_data(fmnist_TEST_URL, root_folder, 'fashion-mnist_test.csv',
                              split_data=False)

print("Train: [{}, {}], Test: [{}, {}]".format(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape))

Train: [(60000, 28, 28, 1), (60000, 1)], Test: [(10000, 28, 28, 1), (10000, 1)]


In [0]:
train_datagen = ImageDataGenerator(width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator()

# test_datagen = ImageDataGenerator()

In [12]:
Y_train_1hot = one_hot_encoder(Y_train).toarray().view(np.float32)
Y_test_1hot = one_hot_encoder(Y_test).toarray().view(np.float32)
# print(Y_train_1hot[0:2])
# print(type(Y_train_1hot))

The labels are: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
The labels are: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [0]:
# plt.imshow(X_train[600,:,:,:])
# plt.title(Y_train[600,0])

In [0]:
class CNN(object):
    def __init__(self, width, height, n_channels, n_classes):

        self.width = width
        self.height = height
        self.channels = n_channels
        self.classes = n_classes
        
        self.model = Sequential()
    
    def compile(self, optimizer, loss, metrics):
        
        #model.add(Reshape(input_shape + (1, ), input_shape=input_shape))
        #self.model.add(Reshape((self.width*self.height*self.channels)+(1,), input_shape=(self.width*self.height*self.channels)))
        #self.model.add(Reshape((self.width, self.height, self.channels), input_shape=(self.width*self.height*self.channels, 1)))
        self.model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation=None, 
                              input_shape=(self.width, self.height, self.channels)))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same'))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(MaxPool2D(pool_size=(2,2)))
        
        self.model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same'))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(MaxPool2D(pool_size=(2,2)))
        
        self.model.add(Flatten())
        self.model.add(Dense(1024, activation='relu'))
        self.model.add(Dropout(rate=0.5))
        self.model.add(Dense(self.classes, activation='softmax'))
        
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)


    def fit(self, X, Y, epochs, batch_size, print_time=None, X_test=None, Y_test=None):

        #n_samples = X.shape[0]
        #X = np.reshape(X, (n_samples, self.width, self.height, self.channels))
        #n_samples = X_test.shape[0]
        #X_test = np.reshape(X_test, (n_samples, self.width, self.height, self.channels))
        Y_1hot = one_hot_encoder(Y).toarray().view(np.float32)
        
        if X_test is None:
            self.model.fit(X, Y_1hot, batch_size=batch_size, epochs=epochs)
        else:
            Y_test_1hot = one_hot_encoder(Y_test).toarray().view(np.float32)
            self.model.fit(X, Y_1hot, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test_1hot))
        


    def predict(self, X, Y, batch_size, return_type='score'):
        #return_type: 'probs', 'score', 'predictions'
        
        if (return_type=='probs'):
            probs = self.model.predict(X)
        elif (return_type=='predictions'):
            return self.model.predict_classes(X)
        elif return_type=='score':
            return self.model.evaluate(X, Y)
        
        return None


In [0]:
#@title Parameters
WIDTH = 28
HEIGHT = 28
N_CHANNELS = 1
N_CLASSES = 10
BATCH_SIZE = 32
MAX_ITER = 10
N_BATCHES = X_train.shape[0]//BATCH_SIZE
PRINT_TIME = N_BATCHES//2
TEST_N_BATCHES = X_test.shape[0]//BATCH_SIZE

In [0]:
ann = CNN(WIDTH, HEIGHT, N_CHANNELS, N_CLASSES)

In [0]:
optimizer = Adam()#SGD(lr=0.001, momentum=0.9, decay=1e-5, nesterov=True)
loss = 'categorical_crossentropy'
ann.compile(optimizer, loss, metrics=['accuracy'])

# Using fit function

In [0]:
history = ann.fit(X_train, Y_train, MAX_ITER, BATCH_SIZE, X_test=X_test, Y_test=Y_test)

In [91]:
n_samples = X_test.shape[0]
X_test_reshaped = np.reshape(X_test, (n_samples, WIDTH, HEIGHT, N_CHANNELS))
predictions = ann.predict(X_test_reshaped, Y_test_1hot, BATCH_SIZE, 'predictions')
print('Accuracy: {}%'.format(np.round(100*(predictions==np.squeeze(Y_test)).sum()/predictions.shape[0], 2)))

Accuracy: 93.08%


# Using ImageDataGenerator

In [0]:
train_datagen.fit(X_train)

In [38]:
history = ann.model.fit_generator(train_datagen.flow(X_train, Y_train_1hot, batch_size=BATCH_SIZE),
                                  epochs=20, validation_data=test_datagen.flow(X_test, Y_test_1hot, batch_size=BATCH_SIZE))

Epoch 1/20
1436/1875 [=====================>........] - ETA: 22s - loss: 6.9270 - acc: 0.3977

1875/1875 [==============================] - 98s 52ms/step - loss: 5.4802 - acc: 0.4733 - val_loss: 0.5889 - val_acc: 0.7784
Epoch 2/20
 479/1875 [======>.......................] - ETA: 1:08 - loss: 0.6996 - acc: 0.7390

1875/1875 [==============================] - 96s 51ms/step - loss: 0.6508 - acc: 0.7561 - val_loss: 0.4445 - val_acc: 0.8304
Epoch 3/20
  97/1875 [>.............................] - ETA: 1:27 - loss: 0.5724 - acc: 0.7838

1782/1875 [===========================>..] - ETA: 4s - loss: 0.5551 - acc: 0.7903

1875/1875 [==============================] - 96s 51ms/step - loss: 0.5532 - acc: 0.7915 - val_loss: 0.4070 - val_acc: 0.8468
Epoch 4/20
 583/1875 [========>.....................] - ETA: 1:04 - loss: 0.5082 - acc: 0.8171

...

1875/1875 [==============================] - 96s 51ms/step - loss: 0.2939 - acc: 0.8932 - val_loss: 0.2379 - val_acc: 0.9117
Epoch 18/20
 597/1875 [========>.....................] - ETA: 1:03 - loss: 0.2817 - acc: 0.9001

1875/1875 [==============================] - 96s 51ms/step - loss: 0.2920 - acc: 0.8947 - val_loss: 0.2308 - val_acc: 0.9129
Epoch 19/20
 132/1875 [=>............................] - ETA: 1:26 - loss: 0.2842 - acc: 0.8970

1774/1875 [===========================>..] - ETA: 5s - loss: 0.2843 - acc: 0.8972

1875/1875 [==============================] - 96s 51ms/step - loss: 0.2851 - acc: 0.8967 - val_loss: 0.2386 - val_acc: 0.9131
Epoch 20/20
 552/1875 [=======>......................] - ETA: 1:06 - loss: 0.2827 - acc: 0.8971

1875/1875 [==============================] - 97s 52ms/step - loss: 0.2825 - acc: 0.8971 - val_loss: 0.2378 - val_acc: 0.9113


In [43]:
# datagen_test = ImageDataGenerator()
# test_generator = datagen_test.flow(
#         test_dir,
#         target_size=(200, 200),
#         color_mode="rgb",
#         shuffle = "false",
#         class_mode='categorical',
#         batch_size=1)

n_samples = X_test.shape[0]
# X_test_reshaped = np.reshape(X_test, (n_samples, WIDTH, HEIGHT, N_CHANNELS))
predictions = ann.model.predict_generator(test_datagen.flow(X_test, Y_test_1hot, batch_size=1, shuffle=False), verbose=True)
predictions = np.argmax(predictions, 1)
print('Accuracy: {}%'.format(np.round(100*(predictions==np.squeeze(Y_test)).sum()/predictions.shape[0], 2)))

10000/10000 [==============================] - 49s 5ms/step
Accuracy: 91.13%


In [42]:
predictions[0:10]

array([[9.97963786e-01, 3.90146554e-10, 1.33788753e-05, 5.93339882e-06,
        5.49025225e-09, 4.12579615e-09, 2.01699580e-03, 4.11917361e-10,
        1.71521393e-08, 1.79212578e-09],
       [8.75908287e-12, 1.00000000e+00, 2.20958408e-12, 6.37593089e-10,
        3.25637503e-11, 4.82809652e-21, 1.45653264e-11, 9.48610477e-23,
        5.02710373e-10, 2.83488693e-18],
       [3.31397132e-05, 1.74146933e-10, 9.93772089e-01, 2.24057408e-06,
        3.64214531e-04, 2.39114761e-12, 5.82821108e-03, 4.06453154e-16,
        6.66596947e-08, 3.54995894e-12],
       [1.84424198e-03, 6.27776422e-03, 3.46613228e-01, 1.62970915e-03,
        2.73381233e-01, 3.42238508e-07, 3.67266804e-01, 2.86537571e-09,
        2.98609701e-03, 6.10252755e-07],
       [4.03676677e-05, 8.70618260e-06, 2.60397233e-03, 9.62836623e-01,
        3.42917591e-02, 4.97374764e-09, 2.05359858e-04, 2.96359278e-14,
        1.31715069e-05, 1.45480685e-11],
       [1.28971159e-01, 1.17388714e-04, 3.32990706e-01, 3.58745828e-02,
   

In [26]:
history = ann.model.fit(X_train, Y_train_1hot, epochs=MAX_ITER, batch_size=BATCH_SIZE, validation_data=(X_test, Y_test_1hot))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
43744/60000 [====================>.........] - ETA: 17s - loss: 0.2355 - acc: 0.9133

60000/60000 [==============================] - 68s 1ms/step - loss: 0.2327 - acc: 0.9151 - val_loss: 0.1990 - val_acc: 0.9276
Epoch 2/10
22624/60000 [==========>...................] - ETA: 40s - loss: 0.2116 - acc: 0.9249

60000/60000 [==============================] - 68s 1ms/step - loss: 0.2081 - acc: 0.9249 - val_loss: 0.1846 - val_acc: 0.9321
Epoch 3/10
14432/60000 [======>.......................] - ETA: 50s - loss: 0.1867 - acc: 0.9318

60000/60000 [==============================] - 69s 1ms/step - loss: 0.1904 - acc: 0.9302 - val_loss: 0.1785 - val_acc: 0.9339
Epoch 4/10
11488/60000 [====>.........................] - ETA: 53s - loss: 0.1813 - acc: 0.9363

60000/60000 [==============================] - 69s 1ms/step - loss: 0.1797 - acc: 0.9346 - val_loss: 0.1829 - val_acc: 0.9317
Epoch 5/10
10336/60000 [====>.........................] - ETA: 54s - loss: 0.1629 - acc: 0.9406

...

60000/60000 [==============================] - 69s 1ms/step - loss: 0.1592 - acc: 0.9424 - val_loss: 0.1698 - val_acc: 0.9375
Epoch 7/10
 9632/60000 [===>..........................] - ETA: 55s - loss: 0.1484 - acc: 0.9481

60000/60000 [==============================] - 69s 1ms/step - loss: 0.1501 - acc: 0.9449 - val_loss: 0.1719 - val_acc: 0.9378
Epoch 8/10
 9568/60000 [===>..........................] - ETA: 55s - loss: 0.1415 - acc: 0.9516

60000/60000 [==============================] - 69s 1ms/step - loss: 0.1433 - acc: 0.9477 - val_loss: 0.1622 - val_acc: 0.9406
Epoch 9/10
 9632/60000 [===>..........................] - ETA: 55s - loss: 0.1347 - acc: 0.9483

60000/60000 [==============================] - 69s 1ms/step - loss: 0.1346 - acc: 0.9505 - val_loss: 0.1611 - val_acc: 0.9403
Epoch 10/10
 9504/60000 [===>..........................] - ETA: 55s - loss: 0.1238 - acc: 0.9548

60000/60000 [==============================] - 69s 1ms/step - loss: 0.1295 - acc: 0.9525 - val_loss: 0.1593 - val_acc: 0.9396


In [0]:
!kill -9 -1